In [2]:
import geopandas as gpd
from shapely.geometry import Point, Polygon
import pandas as pd
import numpy as np
import math
import ray
import time
start_time = time.time()

In [3]:
geom=gpd.read_file('geometry/220527-VOLPE Clean12.dxf')

In [4]:
# HELPER FUNCTION TO ADD POINTS TO THE NODES TABLE
def coordinates_to_nodes(coordinates,id,typeOfEntity=None,entityHandle=None):
    dataframe = pd.DataFrame({
        'id':[str(id)],
        'x':[float(coordinates[0])],
        'y':[float(coordinates[1])],
        'z':[float(coordinates[2])],
        'type_of_entity':[str(typeOfEntity)],
        'EntityHandle':[str(entityHandle)]
        })
    return dataframe

In [5]:
# HELPER FUNCTION TO ADD EDGES TO THE LINKS TABLE
def create_edge(id_prev_p,id_curr_p,id,distance):
    dataframe = pd.DataFrame({
        'id':[str(id)],
        'from':[str(id_prev_p)],
        'to':[str(id_curr_p)],
        'distance':[float(distance)]
    })
    return dataframe

In [6]:
# CHECK IF A POINT IS ALREADY IN A DATAFRAME AND RETURN THE POINT IN THE DATAFRAME IF TRUE 
def check_point(coordinates,original_data_frame):
    for node in original_data_frame.values:
        x_range = [node[1]-0.001,node[1]+0.001]
        y_range = [node[2]-0.001,node[2]+0.001]
        z_range = [node[3]-0.001,node[3]+0.001]
        x_condition = coordinates[0]<x_range[0] or coordinates[0]>x_range[1]
        y_condition = coordinates[1]<y_range[0] or coordinates[1]>y_range[1]
        z_condition = coordinates[2]<z_range[0] or coordinates[2]>z_range[1]
        green_light = x_condition or y_condition or z_condition
        if green_light == False:
            return node,True
    return None, False


In [7]:
# CHECK IF A POINT IS ALREADY IN ONE OF THE POLYGONS OF A DATAFRAME AND RETURN THE POINT IN THE DATAFRAME IF TRUE 
def check_point_polygons(coordinates,geom_polygons):
    for index_p, element_p in enumerate(geom_polygons['geometry']):
        if (Point(coordinates).within(element_p)):
            return True,index_p
    return False,None

In [8]:
# GET DISTANCE BETWEEN A PAIR OF COORDINATES
def get_distance(coords_from,coords_to):
    distance = math.sqrt((coords_from[0] - coords_to[0])**2 + (coords_from[1] - coords_to[1])**2 + (coords_from[2] - coords_to[2])**2)
    distance = distance
    return distance

In [9]:
# FUNCTION TO ADD NODES AND EDGES PER FLOOR
def parse_story(geom_story,story):
    id_story = str(story) # The story id will be used to create a unique identifier per node
    print("Story being evaluated: {}".format(story))
    if len(geom_story['geometry'])<1: 
        print("Story number {} has no geometry".format(story)) # This could be an error with the heights when putting the different stories in the list
        return None, None
    # CREATE GEOMETRIES FOR LAND USES, ALLEYS AND PATHS, USING THE DEFINED SETS
    geom_landUses_complete = geom_story[geom_story['Layer'].apply(lambda x : x in set_landUses)]
    geom_landUses = geom_landUses_complete[geom_landUses_complete['geometry'].apply(lambda x : len(x.coords)>2)]
    geom_alleys = geom_story[geom_story['Layer'].apply(lambda x : x in set_alleys)]
    geom_paths = geom[geom['Layer'].apply(lambda x: x in set_paths)]
    # CHANGE LINETRINGS FOR POLYGONS IN LANDUSES, NECESSARY TO CHECK IF THERE ARE POINTS INSIDE THOSE POLYGONS AND GET ENTRANCES, ETC.
    geom_landUses['geometry'] =  geom_landUses['geometry'].apply(lambda x: Polygon(x.coords) if len(x.coords)>2 else x)
    geom_vertical_mobility = geom_landUses[geom_landUses['Layer'].apply(lambda x : x in set_vertical_mobility)]
    # GET POINTS FOR PATHS IN THE CURRENT STORY
    if(len(geom_paths)<1): print("Causing error, no paths in geometry ") # If there are not paths in the geometry, floors will be disconnected
    points_geom_paths_story = [z for x in geom_paths['geometry'] for z in x.coords if int(round(z[2],0))==(story*13)]
    if(len(points_geom_paths_story)<1): print("Causing error, no paths for story {}".format(story)) # If there are not paths in the current story, it will be disconnected
    not_contained_paths = [] # List used to identify those paths not falling in any mobility polygon (e.g. elevator, stair)
    nodes_df = pd.DataFrame({
    'id':[],
    'x':[],
    'y':[],
    'z':[],
    'type_of_entity':[],
    'EntityHandle':[]
    })
    edges_df = pd.DataFrame({
        'id':[],
        'from':[],
        'to':[],
        'distance':[]
    })
    # ADD ALLEYS TO THE LIST
    for index in np.arange(len(geom_alleys)):
        element_to_evaluate = geom_alleys['geometry'].iloc[index].coords # We get the coordinates of those points composing that alley
        for index_e,point_e in enumerate(element_to_evaluate):
            # EVERY NODE OF THE ELEMENT IS CHECKED TO BE IN THE NODES_DF LIST
            node, node_in_list = check_point(point_e,nodes_df)
            #ADD NODE TO THE NODES_DF LIST IF NOT YET CONTAINED
            if not node_in_list: 
                id_curr_p =  id_story + 'P' + str(len(nodes_df)) # The 'P' is added to make sure that identifiers are unique when adding different stories
                # NODES ARE CHECKED TO BE IN ANY OF THE POLYGONS
                contained_in_p,index_p = check_point_polygons(point_e,geom_landUses) # If it is inside a polygon, it will be an entrance
                entity_handler_container = None
                if(contained_in_p):
                    entity_handler_container = geom_landUses['EntityHandle'].iloc[index_p]
                    nodes_df = pd.concat([nodes_df,coordinates_to_nodes(point_e,id_curr_p,typeOfEntity='Entrance',entityHandle=entity_handler_container)], ignore_index = True)
                else:
                    nodes_df = pd.concat([nodes_df,coordinates_to_nodes(point_e,id_curr_p,typeOfEntity='Alley',entityHandle=entity_handler_container)], ignore_index = True)
            #IF NODE IS ALREADY CONTAINED, TAKE ITS ID
            else: id_curr_p = node[0]
            #FOR EVERY LINE COMPOSING THE ALLEY, CREATE A LINK.
            if index_e > 0: # Links will start to be created from the second point of the alley on
                point_from = nodes_df.loc[nodes_df['id']==str(id_prev_p)]
                coords_from = [point_from['x'].values[0],point_from['y'].values[0],point_from['z'].values[0]]
                if(point_e == None) or (coords_from==None): print("PRONE TO ERROR")
                distance = get_distance(coords_from,point_e)
                edges_df = pd.concat([edges_df,create_edge(id_prev_p,id_curr_p,id_story+ 'P' + str(len(edges_df)),distance)], ignore_index = True)
            #SAVE THE INDEX TO CREATE THE LINK WITH THE NEXT POINT OF THE ALLEY
            id_prev_p = id_curr_p
    # ADD VIRTUAL NODES FOR POLYGONS AND CONNECT TO ENTRANCES
    for index in np.arange(len(geom_landUses)):
        centroid_coordinates=[]
        centroid_coordinates.extend(geom_landUses['geometry'].iloc[index].centroid.coords[0])
        centroid_coordinates.append(geom_landUses['geometry'].iloc[index].exterior.coords[0][2])
        id_curr_p =  id_story + 'P' + str(len(nodes_df)) # The 'P' is added to make sure that identifiers are unique when adding different stories
        entityHandle = geom_landUses['EntityHandle'].iloc[index]
        nodes_df = pd.concat([nodes_df,coordinates_to_nodes(centroid_coordinates,id_curr_p,typeOfEntity=geom_landUses['Layer'].iloc[index],entityHandle=entityHandle)], ignore_index = True)
        #GET ENTRANCES FOR THAT ENTITY AND CREATE LINKS FROM VIRTUAL NODES TO ENTRANCES
        entrances = [node[0] for node in nodes_df.values if node[5]==entityHandle and node[4]=='Entrance']
        if(len(entrances)<1): 
            entityNoEntrance = nodes_df[nodes_df['EntityHandle']==entityHandle]
            print("No entrances for: {} ".format(entityNoEntrance)) # An entity without entrance will be disconnected
        for id in entrances:
            edges_df = pd.concat([edges_df,create_edge(id,id_curr_p,id_story + 'P' + str(len(edges_df)),0)], ignore_index = True) # The 'P' is added to make sure that identifiers are unique when adding different stories
    # ADD PATHS TO THE LIST
    for index,coords in enumerate(points_geom_paths_story):
        contained_in_p,index_p = check_point_polygons(coords,geom_vertical_mobility)
        entity_handler_container = None
        id_curr_p =  id_story + 'P' + str(len(nodes_df)) # The 'P' is added to make sure that identifiers are unique when adding different stories
        if(contained_in_p):
            entity_handler_container = geom_vertical_mobility['EntityHandle'].iloc[index_p]
            #GET ID OF VIRTUAL NODE CORRESPONDING TO THAT ENTITY
            id_entity_list = [node[0] for node in nodes_df.values if node[5]==entity_handler_container and node[4] in set_vertical_mobility]
            id_entity = id_entity_list[0]
            if(id_entity==None): print("id_entity not found for: {}".format(entity_handler_container)) # If the node falls into a polygon, there should be a virtual node for that polygon
            edges_df = pd.concat([edges_df,create_edge(id_curr_p,id_entity,id_story + 'P' + str(len(edges_df)),0)], ignore_index = True)
            nodes_df = pd.concat([nodes_df,coordinates_to_nodes(coords,id_curr_p,typeOfEntity='path',entityHandle=entity_handler_container)], ignore_index = True)
        else:
            not_contained_paths.append(coords)
    if(len(not_contained_paths)>1): print("Non contained paths for story {} are: {}".format(story,not_contained_paths)) # This means there are paths not contained in polygons for a certain story, which should not happen 
    return nodes_df,edges_df

In [10]:
# CREATE NEEDED SETS
set_alleys = {'ALLEY-ELEVATOR','ALLEY1'}
set_paths = {'STAIRS-PATH','ELEVATORS-PATH'} # Vertical links among floors
set_vertical_mobility = {'STAIRS', 'ELEVATORS'}
set_landUses = {'STAIRS', 'ELEVATORS','PARKS','OFFICES','BANKS','LIBRARY','HEALTH_CARE','LAUNDRY','RESTAURANT','CAFE','BAR','SCHOOL','CONVENIENCE_STORE','GROCERY_OR_SUPERMARKET','HOME_GOODS_STORE',
'FURNITURE_STORE','PHARMACY','PROFESSIONAL_SERVICES','LEISURE_AND_WELLNESS','FAMILY_OCCUPANCY','DUAL__OCCUPANCY','TRIPLE_OCCUPANCY',
'QUAD_OCCUPANCY','SINGLE_OCCUPANCY'}

In [11]:
#GET LIST OF GEOMETRIES PER FLOOR
list_geoms = [] # List that will contain the geometries of each floor
nbr_stories = 52 # Number of stories that will be covered by nodes
heigh_per_story = 13 # Separation between stories (same unity as in CAD)
for z in np.arange(nbr_stories):
    geom_story = geom[geom['geometry'].apply(lambda x : round(x.coords[0][2],0) == z*heigh_per_story)]
    list_geoms.append(geom_story)

In [12]:
#WE FIRST SHUTDOWN THE PARALLELIZATION MODULE TO MAKE SURE IT IS NOT TAKING COMPUTING RESOURCES ALREADY
ray.shutdown()

In [13]:
# Ray task is created for parallelization among parsing floors
remote_parse_story = ray.remote(parse_story)

In [14]:
ray_object = ray.get([remote_parse_story.remote(list_geoms[story],story) for story in range(0,len(list_geoms))])

 pid=18808) Story being evaluated: 2
 pid=8016) Story being evaluated: 8
 pid=372) Story being evaluated: 3
 pid=4584) Story being evaluated: 4
 pid=8596) Story being evaluated: 12
 pid=17496) Story being evaluated: 6
 pid=9160) Story being evaluated: 10
 pid=18620) Story being evaluated: 1
 pid=13144) Story being evaluated: 11
 pid=15712) Story being evaluated: 9
 pid=5712) Story being evaluated: 14
 pid=8828) Story being evaluated: 7
 pid=17772) Story being evaluated: 5
 pid=18600) Story being evaluated: 0
 pid=16736) Story being evaluated: 15
 pid=15512) Story being evaluated: 13


 pid=8016) c:\Users\adminlocal\anaconda3\envs\verticalCity\lib\site-packages\geopandas\geodataframe.py:1322: SettingWithCopyWarning: 
 pid=8016) A value is trying to be set on a copy of a slice from a DataFrame.
 pid=8016) Try using .loc[row_indexer,col_indexer] = value instead
 pid=8016) _story
 pid=8016) See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
 pid=8016)   super(GeoDataFrame, self).__setitem__(key, value)
 pid=18808) c:\Users\adminlocal\anaconda3\envs\verticalCity\lib\site-packages\geopandas\geodataframe.py:1322: SettingWithCopyWarning: 
 pid=18808) A value is trying to be set on a copy of a slice from a DataFrame.
 pid=18808) Try using .loc[row_indexer,col_indexer] = value instead
 pid=18808) story
 pid=18808) See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
 pid=18808)   super(GeoDataFrame, self)

 pid=18808) Story being evaluated: 16
 pid=372) Story being evaluated: 17
 pid=4584) Story being evaluated: 19
 pid=17496) Story being evaluated: 18


 pid=18808) c:\Users\adminlocal\anaconda3\envs\verticalCity\lib\site-packages\geopandas\geodataframe.py:1322: SettingWithCopyWarning: 
 pid=18808) A value is trying to be set on a copy of a slice from a DataFrame.
 pid=18808) Try using .loc[row_indexer,col_indexer] = value instead
 pid=18808) story
 pid=18808) See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
 pid=18808)   super(GeoDataFrame, self).__setitem__(key, value)
 pid=372) c:\Users\adminlocal\anaconda3\envs\verticalCity\lib\site-packages\geopandas\geodataframe.py:1322: SettingWithCopyWarning: 
 pid=372) A value is trying to be set on a copy of a slice from a DataFrame.
 pid=372) Try using .loc[row_indexer,col_indexer] = value instead
 pid=372) e_story
 pid=372) See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
 pid=372)   super(GeoDataFrame, self).__se

 pid=18620) Story being evaluated: 20
 pid=17772) Story being evaluated: 21
 pid=18600) Story being evaluated: 22


...

 pid=9160) c:\Users\adminlocal\anaconda3\envs\verticalCity\lib\site-packages\geopandas\geodataframe.py:1322: SettingWithCopyWarning: 
 pid=9160) A value is trying to be set on a copy of a slice from a DataFrame.
 pid=9160) Try using .loc[row_indexer,col_indexer] = value instead
 pid=9160) _story
 pid=9160) See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
 pid=9160)   super(GeoDataFrame, self).__setitem__(key, value)


 pid=17772) Story being evaluated: 48
 pid=18620) Story being evaluated: 49


 pid=17772) c:\Users\adminlocal\anaconda3\envs\verticalCity\lib\site-packages\geopandas\geodataframe.py:1322: SettingWithCopyWarning: 
 pid=17772) A value is trying to be set on a copy of a slice from a DataFrame.
 pid=17772) Try using .loc[row_indexer,col_indexer] = value instead
 pid=17772) story
 pid=17772) See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
 pid=17772)   super(GeoDataFrame, self).__setitem__(key, value)
 pid=18620) c:\Users\adminlocal\anaconda3\envs\verticalCity\lib\site-packages\geopandas\geodataframe.py:1322: SettingWithCopyWarning: 
 pid=18620) A value is trying to be set on a copy of a slice from a DataFrame.
 pid=18620) Try using .loc[row_indexer,col_indexer] = value instead
 pid=18620) story
 pid=18620) See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
 pid=18620)   super(GeoDataFrame, 

 pid=17496) Story being evaluated: 50
 pid=18808) Story being evaluated: 51


 pid=17496) c:\Users\adminlocal\anaconda3\envs\verticalCity\lib\site-packages\geopandas\geodataframe.py:1322: SettingWithCopyWarning: 
 pid=17496) A value is trying to be set on a copy of a slice from a DataFrame.
 pid=17496) Try using .loc[row_indexer,col_indexer] = value instead
 pid=17496) story
 pid=17496) See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
 pid=17496)   super(GeoDataFrame, self).__setitem__(key, value)
 pid=18808) c:\Users\adminlocal\anaconda3\envs\verticalCity\lib\site-packages\geopandas\geodataframe.py:1322: SettingWithCopyWarning: 
 pid=18808) A value is trying to be set on a copy of a slice from a DataFrame.
 pid=18808) Try using .loc[row_indexer,col_indexer] = value instead
 pid=18808) story
 pid=18808) See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
 pid=18808)   super(GeoDataFrame, 

In [15]:
#CREATE THE NODES AND EDGES TABLE AND ADD THE PARSED STORIES
nodes_df = pd.DataFrame({
    'id':[],
    'x':[],
    'y':[],
    'z':[],
    'type_of_entity':[],
    'EntityHandle':[]
    })
edges_df = pd.DataFrame({
    'id':[],
    'from':[],
    'to':[],
    'distance':[]
})

for element in ray_object:
    element[0]
    nodes_df = pd.concat([nodes_df,element[0]], ignore_index = True)
    edges_df = pd.concat([edges_df,element[1]], ignore_index = True)

In [16]:
# ADD THOSE EDGES CORRESPONDING TO THE PATHS LINKING EDGES
path_nodes = nodes_df[nodes_df['type_of_entity'].apply(lambda x: x=='path')]
geom_paths = geom[geom['Layer'].apply(lambda x: x in set_paths)]
for index in np.arange(len(geom_paths)):
    print("{} of {} paths evaluated".format(index,len(geom_paths)))     
    element_to_evaluate = geom_paths['geometry'].iloc[index].coords
    for index_e,point_e in enumerate(element_to_evaluate):
        # EVERY NODE OF THE ELEMENT IS CHECKED TO BE IN THE NODES_DF LIST
        node, node_in_list = check_point(point_e,path_nodes)
        if not node_in_list:
            print("Point_e is: {}".format(point_e)) # every path node should have been added already to the list
            print("NODE NOT FOUND")
        #IF NODE IS ALREADY CONTAINED, TAKE ITS ID
        else:
            id_curr_p = node[0]
        #FOR EVERY LINE COMPOSING THE PATH, CREATE A LINK.
        if index_e > 0:
            point_from = nodes_df.loc[nodes_df['id']==str(id_prev_p)]
            coords_from = [point_from['x'].values[0],point_from['y'].values[0],point_from['z'].values[0]]
            distance = get_distance(coords_from,point_e)
            edges_df = pd.concat([edges_df,create_edge(id_prev_p,id_curr_p,'VL'+str(len(edges_df)),distance)], ignore_index = True)
        #SAVE THE INDEX TO CREATE THE EDGE WITH THE NEXT POINT OF THE PATH
        id_prev_p = id_curr_p

0 of 83 paths evaluated
1 of 83 paths evaluated
2 of 83 paths evaluated
3 of 83 paths evaluated
4 of 83 paths evaluated
5 of 83 paths evaluated
6 of 83 paths evaluated
7 of 83 paths evaluated
8 of 83 paths evaluated
9 of 83 paths evaluated
10 of 83 paths evaluated
11 of 83 paths evaluated
12 of 83 paths evaluated
13 of 83 paths evaluated
14 of 83 paths evaluated
15 of 83 paths evaluated
16 of 83 paths evaluated
17 of 83 paths evaluated
18 of 83 paths evaluated
19 of 83 paths evaluated
20 of 83 paths evaluated
21 of 83 paths evaluated
22 of 83 paths evaluated
23 of 83 paths evaluated
24 of 83 paths evaluated
25 of 83 paths evaluated
26 of 83 paths evaluated
27 of 83 paths evaluated
28 of 83 paths evaluated
29 of 83 paths evaluated
30 of 83 paths evaluated
31 of 83 paths evaluated
32 of 83 paths evaluated
33 of 83 paths evaluated
34 of 83 paths evaluated
35 of 83 paths evaluated
36 of 83 paths evaluated
37 of 83 paths evaluated
38 of 83 paths evaluated
39 of 83 paths evaluated
40 of 83 p

In [17]:
# SAVE THE NODES AND LINK TABLES IN CSV FILES
nodes_df.to_csv('nodes_df.csv')
edges_df.to_csv('edges_df.csv')

In [18]:
print("--- {} seconds ---".format(time.time() - start_time))

--- 119.55061173439026 seconds ---


## DEBUGGING STUFF

In [ ]:
import geopandas as gpd

In [ ]:
geom=gpd.read_file('geometry/220527-VOLPE Clean12.dxf')
story = 31
set_paths = {'STAIRS-PATH','ELEVATORS-PATH'}
geom_paths = geom[geom['Layer'].apply(lambda x: x in set_paths)]



In [ ]:
geom_paths

In [ ]:
[z for x in geom_paths['geometry'] for z in x.coords if int(round(z[2],0))==(story*13)]

In [ ]:
for x in geom_paths['geometry']:
    for z in x.coords:
        print(z)

In [ ]:
geom_story = list_geoms[51]
geom_landUses_complete = geom_story[geom_story['Layer'].apply(lambda x : x in set_landUses)]
geom_landUses = geom_landUses_complete[geom_landUses_complete['geometry'].apply(lambda x : len(x.coords)>2)]
geom_landUses['geometry'] =  geom_landUses['geometry'].apply(lambda x: Polygon(x.coords) if len(x.coords)>2 else x)
geom_alleys = geom_story[geom_story['Layer'].apply(lambda x : x in set_alleys)]
geom_vertical_mobility = geom_landUses[geom_landUses['Layer'].apply(lambda x : x in set_vertical_mobility)]
geom_paths = geom[geom['Layer'].apply(lambda x: x in set_paths)]
# CHANGE LINETRINGS FOR POLYGONS LANDUSES
points_geom_paths_story = [z for x in geom_paths['geometry'] for z in x.coords if int(round(z[2],0))==(51*13)]
not_contained_paths = []
for index,coords in enumerate(points_geom_paths_story):
        #if index%10 == 0: print("{} of {} elements evaluated".format(index,len(points_geom_paths_story)))     
        contained_in_p,index_p = check_point_polygons(coords,geom_vertical_mobility)
        entity_handler_container = None
        if(contained_in_p):
            entity_handler_container = geom_landUses['EntityHandle'].iloc[index_p]
        else: 
            #print("Path not contained: {}".format(coords))
            not_contained_paths.append(coords)
if(len(not_contained_paths)>1): print("Non contained paths for story {} are: {}".format(51,not_contained_paths))

In [ ]:
geom_vertical_mobility.values[index_p]

In [ ]:
# CREATE GEOMETRIES
geom_story = list_geoms[51]
geom_landUses_complete = geom_story[geom_story['Layer'].apply(lambda x : x in set_landUses)]
geom_landUses = geom_landUses_complete[geom_landUses_complete['geometry'].apply(lambda x : len(x.coords)>2)]
geom_landUses['geometry'] =  geom_landUses['geometry'].apply(lambda x: Polygon(x.coords) if len(x.coords)>2 else x)
geom_alleys = geom_story[geom_story['Layer'].apply(lambda x : x in set_alleys)]
geom_vertical_mobility = geom_landUses[geom_landUses['Layer'].apply(lambda x : x in set_vertical_mobility)]
contained_in_p,index_p = check_point_polygons([768850.8833876338, 2958449.378691245, 663.0],geom_vertical_mobility)
if(contained_in_p): print(geom_vertical_mobility.values[index_p])

In [ ]:
# CREATE GEOMETRIES
geom_story = list_geoms[51]
geom_landUses_complete = geom_story[geom_story['Layer'].apply(lambda x : x in set_landUses)]
geom_landUses = geom_landUses_complete[geom_landUses_complete['geometry'].apply(lambda x : len(x.coords)>2)]
geom_landUses['geometry'] =  geom_landUses['geometry'].apply(lambda x: Polygon(x.coords) if len(x.coords)>2 else x)
geom_alleys = geom_story[geom_story['Layer'].apply(lambda x : x in set_alleys)]
# ADD ALLEYS TO THE LIST
for index in np.arange(len(geom_alleys)):
    #if index%10 == 0: print("{} of {} points from alleys evaluated".format(index,len(geom_alleys)))     
    element_to_evaluate = geom_alleys['geometry'].iloc[index].coords
    for index_e,point_e in enumerate(element_to_evaluate):
        # NODES ARE CHECKED TO BE IN ANY OF THE POLYGONS
        geom_stairs_evaluate = geom_landUses[geom_landUses['EntityHandle']=='95CCE']
        contained_in_p,index_p = check_point_polygons(point_e,geom_stairs_evaluate)
        entity_handler_container = None
        if(contained_in_p):
            print(point_e)

In [ ]:
geom_paths = geom[geom['Layer'].apply(lambda x: x in set_paths)]
geom_paths_51 = geom_paths[geom_paths['geometry'].apply(lambda x: x.coords[0])]
points_geom_paths_story = [z for x in geom_paths['geometry'] for z in x.coords if int(round(z[2],0))==(51*13)]


In [ ]:
geom_paths['geometry']

In [ ]:
set_not_found = {768205.4923148999,768213.401961236}

In [ ]:
points_not_found = geom_paths[geom_paths['geometry'].apply(lambda x: x.coords[0])]

In [ ]:
#ALLEY ELEVATOR 95D16
geom_95D16 = geom[geom['EntityHandle']=='95D16']
#geom_elevator_evaluate = geom_landUses[geom_landUses['EntityHandle']=='93948']

In [ ]:
for element in geom_95D16['geometry'].iloc[0].coords:
    contained_in_p,index_p = check_point_polygons(element,geom_landUses)
    if(contained_in_p): print(geom_landUses.values[index_p])